In [1]:
import numpy as np
import pandas as pd

In [24]:
df = pd.read_csv('../static/data/movie_test.csv')
df.code = df.code.astype(str)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)

(2746, 17)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\r\n사랑하는...",장편,예술영화,SF,프랑스,97분 57초,12세이상관람가,가가린 옥상 에서 하늘 보며 우주비행사 꿈꾸던 10 대 소년 유리 사랑 하는 사람 ...


In [25]:
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.m_genre) * 3 + (' ' + df.movie_director) * 2 + (' ' + df.star_actor) * 2

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(2746, 38694)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
total_cv = cvect.fit_transform(df.total)
total_cv.shape

(2746, 38694)

In [28]:
indices = pd.Series(df.index, index=df.code)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)
cosine_sim.shape

(2746, 2746)

In [30]:
cosine_sim_cv = cosine_similarity(total_cv)
cosine_sim_cv.shape

(2746, 2746)

In [31]:
def get_recommendation(code, cos_sim):
    index = indices[code]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.code.iloc[movie_indices], df.title.iloc[movie_indices]

In [32]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
cosine_sim_total.shape

(2746, 2746)

In [33]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total_cv = linear_kernel(total_cv, total_cv)
cosine_sim_total_cv.shape

(2746, 2746)

In [34]:
df.code

0       20210397
1       20209771
2       20192817
3       20163603
4       20231496
          ...   
2741    19980176
2742    20227756
2743    20207222
2744    20188441
2745    20194403
Name: code, Length: 2746, dtype: object

In [35]:
code = "20112018"

In [36]:
codes, titles = get_recommendation(code, cosine_sim_cv)
codes, titles

(984     20223268
 615     20200627
 312     20147527
 42      20080513
 1423    20198291
 866     20228854
 27      20208616
 74      20231009
 1081    20205001
 536     20227066
 Name: code, dtype: object,
 984                        더 컨테이젼: 바이러스의 습격 (The Leakers)
 615                             나의 끝, 당신의 시작 (Relativity)
 312     그날 본 꽃의 이름을 우리는 아직 모른다 (Anohana: The Flower We...
 42                     갓파 쿠와 여름방학을 (Summer Days With Coo)
 1423                                      로켓맨 (Rocketman)
 866                              단청낭자 (Dan Qing Xiao Jie)
 27                                                    간이역
 74                거울 속 외딴 성 (Lonely Castle in the Mirror)
 1081                           도망친 여자 (The Woman Who Ran)
 536                                           깐부(코로나방학소동)
 Name: title, dtype: object)

In [37]:
get_recommendation(code, cosine_sim)

(2186    20181404
 258     20217573
 984     20223268
 521     20212281
 2189    20174369
 74      20231009
 1321    20127526
 504     20211305
 746     20194967
 603     20175587
 Name: code, dtype: object,
 2186    벽 속에 숨은 마법시계 (The house with a clock in its wa...
 258                   굴뚝마을의 푸펠 (Poupelle of Chimney Town)
 984                        더 컨테이젼: 바이러스의 습격 (The Leakers)
 521                                                  긴 하루
 2189                           변산 (Sunset in My Hometown)
 74                거울 속 외딴 성 (Lonely Castle in the Mirror)
 1321                           러시안 소설 (The Russian Novel)
 504                                      기문환생 (My Lord 1)
 746                                 네레우스: 바다의 괴물 (Nereus)
 603                                 나비잠 (Butterfly Sleep)
 Name: title, dtype: object)

In [38]:
# for code in df.code.values:
sim_scores = pd.Series(cosine_sim[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

2186    0.094314
258     0.093134
984     0.092703
521     0.092205
2189    0.091780
74      0.089512
1321    0.087183
504     0.086857
746     0.086163
dtype: float64

In [39]:
sim_scores = pd.Series(cosine_sim_cv[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

984     0.262949
615     0.256022
312     0.250270
42      0.239766
1423    0.238476
866     0.232574
27      0.229989
74      0.228408
1081    0.223267
dtype: float64

In [40]:
codes

984     20223268
615     20200627
312     20147527
42      20080513
1423    20198291
866     20228854
27      20208616
74      20231009
1081    20205001
536     20227066
Name: code, dtype: object

In [41]:
df[df.code.isin(codes)][['title']]

,title
27,간이역
42,갓파 쿠와 여름방학을 (Summer Days With Coo)
74,거울 속 외딴 성 (Lonely Castle in the Mirror)
312,그날 본 꽃의 이름을 우리는 아직 모른다 (Anohana: The Flower We...
536,깐부(코로나방학소동)
615,"나의 끝, 당신의 시작 (Relativity)"
866,단청낭자 (Dan Qing Xiao Jie)
984,더 컨테이젼: 바이러스의 습격 (The Leakers)
1081,도망친 여자 (The Woman Who Ran)
1423,로켓맨 (Rocketman)
